In [3]:
import time
import networkx as nx
import numpy as np
from pathlib import Path
import scipy as sp
rng = np.random.default_rng()
from collections import deque

In [4]:
gname = "ego-facebook"

gnn_path = Path(f"results/gnn/{gname}_50.mtx")
slicer_path = Path(f"results/slicer/{gname}_50.mtx")
actual_path = Path(f"datasets/real/{gname}/graph.mtx")

imm_gnn = Path(f"IMM/{gname}-gnn/")
imm_slicer= Path(f"IMM/{gname}-slicer/")
imm_actual = Path(f"IMM/{gname}-actual/")

with open(gnn_path, "rb") as fh:
  G_gnn = nx.from_scipy_sparse_array(sp.io.mmread(fh), create_using=nx.DiGraph)
with open(slicer_path, "rb") as fh:
  G_slicer = nx.from_scipy_sparse_array(sp.io.mmread(fh), create_using=nx.DiGraph)
with open(actual_path, "rb") as fh:
  G_actual = nx.from_scipy_sparse_array(sp.io.mmread(fh), create_using=nx.DiGraph)

In [5]:
with open(imm_gnn / "attribute.txt", "w") as fh:
    fh.write(f"n={G_gnn.number_of_nodes()}\n")
    fh.write(f"m={G_gnn.number_of_edges()}\n")
with open(imm_gnn / "graph.txt", "wb") as fh:
    nx.write_edgelist(G_gnn, fh, data=False)
with open(imm_gnn / "graph_ic.inf", "wb") as fh:
    nx.write_edgelist(G_gnn, fh, data=['weight'])

with open(imm_slicer / "attribute.txt", "w") as fh:
    fh.write(f"n={G_slicer.number_of_nodes()}\n")
    fh.write(f"m={G_slicer.number_of_edges()}\n")
with open(imm_slicer / "graph.txt", "wb") as fh:
    nx.write_edgelist(G_slicer, fh, data=False)
with open(imm_slicer / "graph_ic.inf", "wb") as fh:
    nx.write_edgelist(G_slicer, fh, data=['weight'])

with open(imm_actual / "attribute.txt", "w") as fh:
    fh.write(f"n={G_actual.number_of_nodes()}\n")
    fh.write(f"m={G_actual.number_of_edges()}\n")
with open(imm_actual / "graph.txt", "wb") as fh:
    nx.write_edgelist(G_actual, fh, data=False)
with open(imm_actual / "graph_ic.inf", "wb") as fh:
    nx.write_edgelist(G_actual, fh, data=['weight'])

In [6]:
def monte_carlo_trial(G: nx.DiGraph, seed_nodes: list[int]):
    activated_nodes = set(seed_nodes)
    queue = deque(seed_nodes)
    while queue:
        node = queue.popleft()
        for neighbor in G.neighbors(node):
            if neighbor not in activated_nodes and rng.random() <= G[node][neighbor]['weight']:
                activated_nodes.add(neighbor)
                queue.append(neighbor)
    return len(activated_nodes)

def monto_carlo_spread(G: nx.DiGraph, seed_nodes: list[int], T: int = 5000):
    total_spread = 0
    for _ in range(T):
        total_spread += monte_carlo_trial(G, seed_nodes)
    return total_spread / T


### eps = 0.1

In [7]:
gnn_seeds = [
    [107],
    [107, 1973,],
    [107, 1839, 2078, 1827,],
    [107, 1831, 3556, 1827, 36, 3687, 3717, 3625,],
    [0, 1831, 2090, 1827, 3531, 1828, 3857, 1513, 2012, 1374, 3905, 3860, 2033, 3535, 2004, 3720,],
    [107, 1864, 3774, 325, 1827, 3687, 4022, 3926, 2012, 4024, 3577, 3857, 3686, 1859, 1513, 3820, 3717, 2018, 1828, 2030, 3574, 4035, 3595, 3994, 3600, 3918, 1837, 2000, 3961, 2682, 3535, 756,],
]

slicer_seeds = [
    [134],
    [0, 1884,],
    [223, 1993, 1827, 2282,],
    [72, 1854, 1827, 2282, 3882, 4021, 1759, 2425,],
    [104, 1983, 1827, 2282, 1792, 4000, 1572, 3655, 1072, 89, 259, 2031, 1857, 597, 2864, 1820,],
    [297, 1851, 1827, 2282, 1742, 4031, 1836, 3682, 4038, 597, 1512, 2596, 91, 2018, 310, 1830, 2528, 2012, 624, 1943, 1572, 102, 1947, 2028, 2344, 1339, 166, 2004, 83, 2030, 190, 591,],
]

actual_seeds = [
    [107],
    [107, 1858,],
    [107, 1864, 3517, 186,],
    [351, 1973, 3492, 25, 1827, 2642, 3652, 1513,],
    [136, 1831, 3851, 277, 1827, 312, 3879, 4031, 2425, 1794, 2016, 3179, 1381, 2442, 2834, 2307,],
    [136, 1831, 3626, 109, 1827, 3761, 162, 2489, 3996, 1513, 2642, 175, 4, 3687, 2016, 4027, 2831, 3717, 1913, 3122, 2010, 3303, 337, 1793, 2033, 2657, 1339, 3905, 3722, 2596, 115, 3882,],
]

In [8]:
spreads = []
for i in range(len(actual_seeds)):
    gnn_spread = monto_carlo_spread(G_actual, gnn_seeds[i], T=1000)
    slicer_spread = monto_carlo_spread(G_actual, slicer_seeds[i], T=1000)
    actual_spread = monto_carlo_spread(G_actual, actual_seeds[i], T=1000)
    spreads.append([gnn_spread, slicer_spread, actual_spread])

In [9]:
from tabulate import tabulate

headers = ['k', 'GNN', 'Slicer', 'Ground Truth']
table = [headers]
for i in range(len(spreads)):
    k = 2**i
    table.append([f"{k}", *spreads[i]])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒═════╤═════════╤══════════╤════════════════╕
│   k │     GNN │   Slicer │   Ground Truth │
╞═════╪═════════╪══════════╪════════════════╡
│   1 │ 2926.05 │  2419.32 │        2921.37 │
├─────┼─────────┼──────────┼────────────────┤
│   2 │ 3030.85 │  2911.36 │        3030.3  │
├─────┼─────────┼──────────┼────────────────┤
│   4 │ 3063.44 │  2957.02 │        3072.5  │
├─────┼─────────┼──────────┼────────────────┤
│   8 │ 3083.88 │  3049.21 │        3090.45 │
├─────┼─────────┼──────────┼────────────────┤
│  16 │ 3101.1  │  3094.92 │        3105.26 │
├─────┼─────────┼──────────┼────────────────┤
│  32 │ 3122.61 │  3112.94 │        3128.86 │
╘═════╧═════════╧══════════╧════════════════╛


### eps = 0.03

In [10]:
gnn_seeds = [
    [809],
    [351, 1987,],
    [107, 1831, 3769, 1827,],
    [107, 1973, 3581, 1827, 25, 4031, 3573, 2031,],
    [464, 1973, 3805, 1827, 9, 4031, 3531, 3857, 2031, 1828, 3955, 4021, 1984, 4002, 3926, 2027,],
    [351, 1831, 2088, 1827, 26, 4031, 3531, 2031, 3857, 1828, 3773, 3913, 4002, 1984, 4021, 4001, 3746, 3833, 3639, 3717, 4017, 488, 3586, 3985, 2040, 3678, 3484, 2020, 3541, 1961, 889, 2024,],
]

slicer_seeds = [
    [104],
    [297, 1992],
    [297, 1882, 1827, 2282,],
    [297, 1945, 1827, 2282, 1742, 3655, 1572, 3985,],
    [297, 1959, 1827, 2282, 1759, 1572, 3985, 3655, 2727, 624, 4028, 19, 259, 2033, 4002, 1947,],
    [297, 1937, 1827, 2282, 1742, 1572, 3655, 4024, 2727, 259, 889, 19, 4028, 1947, 2033, 4002, 3388, 
     3995, 597, 1834, 2864, 3702, 1680, 4020, 2019, 1943, 3700, 624, 327, 198, 321, 1852,]
]

actual_seeds = [
    [107],
    [107, 1831,],
    [107, 1831, 2088, 142,],
    [107, 1831, 1821, 26, 1827, 115, 3575, 4000,],
    [107, 1831, 2055, 0, 1827, 3761, 3652, 2596, 3985, 2489, 326, 1794, 181, 175, 1518, 1934,],
    [107, 1831, 2088, 26, 1827, 312, 3985, 3575, 2489, 3687, 2596, 1794, 99, 181, 1934, 2442, 3722, 1518, 
     2016, 3645, 3995, 2009, 4000, 2031, 319, 3521, 3652, 1680, 2384, 1912, 3179, 624,]
]

In [11]:
spreads = []
for i in range(len(actual_seeds)):
    gnn_spread = monto_carlo_spread(G_actual, gnn_seeds[i], T=1000)
    slicer_spread = monto_carlo_spread(G_actual, slicer_seeds[i], T=1000)
    actual_spread = monto_carlo_spread(G_actual, actual_seeds[i], T=1000)
    spreads.append([gnn_spread, slicer_spread, actual_spread])

from tabulate import tabulate

headers = ['k', 'GNN', 'Slicer', 'Ground Truth']
table = [headers]
for i in range(len(spreads)):
    k = 2**i
    table.append([f"{k}", *spreads[i]])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒═════╤═════════╤══════════╤════════════════╕
│   k │     GNN │   Slicer │   Ground Truth │
╞═════╪═════════╪══════════╪════════════════╡
│   1 │ 2918.35 │  2601.05 │        2923.5  │
├─────┼─────────┼──────────┼────────────────┤
│   2 │ 3027.76 │  2800.85 │        3030.73 │
├─────┼─────────┼──────────┼────────────────┤
│   4 │ 3065.31 │  3000.76 │        3072.25 │
├─────┼─────────┼──────────┼────────────────┤
│   8 │ 3093.38 │  3072.11 │        3093.94 │
├─────┼─────────┼──────────┼────────────────┤
│  16 │ 3103.91 │  3089.46 │        3110.15 │
├─────┼─────────┼──────────┼────────────────┤
│  32 │ 3123.33 │  3120.36 │        3137.84 │
╘═════╧═════════╧══════════╧════════════════╛


In [ ]:
spreads = []
for i in range(len(actual_seeds)):
    T = 100
    gnn_spread = monto_carlo_spread(G_actual, gnn_seeds[i], T)
    slicer_spread = monto_carlo_spread(G_actual, slicer_seeds[i], T)
    actual_spread = monto_carlo_spread(G_actual, actual_seeds[i], T)
    spreads.append([gnn_spread, slicer_spread, actual_spread])

from tabulate import tabulate

headers = ['k', 'GNN', 'Slicer', 'Ground Truth']
table = [headers]
for i in range(len(spreads)):
    k = 2**i
    table.append([f"{k}", *spreads[i]])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))